In [42]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [43]:
import os.path
from pathlib import Path

import json
import pyam
import pandas as pd
import numpy as np

import utils
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

<IPython.core.display.Javascript object>

In [44]:
ID = "sb_noSRM_inclu"

<IPython.core.display.Javascript object>

In [45]:
data = pd.read_csv(
    str(
        Path(
            utils.DATA_DIR
            / "interim"
            / "delta_rf"
            / f"{ID}_delta-erf_meta_TCRE_ratio.csv"
        )
    )
)
# data_ = pd.read_csv(
#     str(Path(utils.DATA_DIR / "interim" / "delta_rf" / f"{ID}_delta-erf.csv"))
# )

OUTPUT_FILE = Path(utils.DATA_DIR / "plotting_data" / "figure2" / f"{ID}.csv")
OUTPUT_FILE_MEDIAN = Path(
    utils.DATA_DIR / "plotting_data" / "figure2" / f"{ID}_median_stats.csv"
)

<IPython.core.display.Javascript object>

Filter out one of the variables (doesn't matter which)

In [46]:
data = data[data["variable"] == "delta_rf"]

<IPython.core.display.Javascript object>

In [47]:
data.drop("run_id", axis=1, inplace=True)

<IPython.core.display.Javascript object>

In [48]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', '2030_warming', '2100_warming',
       'SRM_at_first_crossing', 'TCRE',
       'average_annual_emissions_2030_net-zero',
       'average_cdr_between_net0_and_15', 'climate_model',
       'cumulative_emms_15', 'cumulative_emms_net0_to_15',
       'cumulative_emms_peak_warming', 'cumulative_emms_peak_warming_to_15',
       'cumulative_pos_emms_at_15', 'cumulative_pos_emms_at_net0',
       'ensemble_member', 'max_assigned_CDR', 'max_deployed_CDR', 'model',
       'ms', 'net-zero_year', 'net_CDR_at_15_temp', 'peak_warming', 'region',
       'scenario', 'total_CDR', 'unit', 'variable',
       'year_return_back_below_15_temp', 'TCRE_down', 'TCRE_up',
       'TCRE_ratio_up', 'TCRE_up_minus_down', 'Year of net-zero CO2',
       'Average annual NNEs', 'Cum emissions peak-warming 1.5°C',
       'peak_warming_categories', 'TCRE_ratio_up_cat', 't_above_15', 'ratios'],
      dtype='object')

<IPython.core.display.Javascript object>

In [49]:
#data = data[(data["peak_warming"] < 2.55) & (data["peak_warming"] > 2.45)]


<IPython.core.display.Javascript object>

Filter out pathways that have not reached net-zero by 2500

In [50]:
data = data[
    (data["year_return_back_below_15_temp"] < 2501)
    & (data["year_return_back_below_15_temp"] > 2029)
]

### limit peak warming -> to remove again later?
data = data[(data["peak_warming"] > 1.6)]

<IPython.core.display.Javascript object>

In [51]:
data["year_return_back_below_15_temp"].describe()

count    156919.000000
mean       2227.370229
std         110.531540
min        2046.000000
25%        2137.000000
50%        2203.000000
75%        2302.000000
max        2500.000000
Name: year_return_back_below_15_temp, dtype: float64

<IPython.core.display.Javascript object>

Select relevant columns

In [52]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', '2030_warming', '2100_warming',
       'SRM_at_first_crossing', 'TCRE',
       'average_annual_emissions_2030_net-zero',
       'average_cdr_between_net0_and_15', 'climate_model',
       'cumulative_emms_15', 'cumulative_emms_net0_to_15',
       'cumulative_emms_peak_warming', 'cumulative_emms_peak_warming_to_15',
       'cumulative_pos_emms_at_15', 'cumulative_pos_emms_at_net0',
       'ensemble_member', 'max_assigned_CDR', 'max_deployed_CDR', 'model',
       'ms', 'net-zero_year', 'net_CDR_at_15_temp', 'peak_warming', 'region',
       'scenario', 'total_CDR', 'unit', 'variable',
       'year_return_back_below_15_temp', 'TCRE_down', 'TCRE_up',
       'TCRE_ratio_up', 'TCRE_up_minus_down', 'Year of net-zero CO2',
       'Average annual NNEs', 'Cum emissions peak-warming 1.5°C',
       'peak_warming_categories', 'TCRE_ratio_up_cat', 't_above_15', 'ratios'],
      dtype='object')

<IPython.core.display.Javascript object>

In [53]:
# data_sel = data_netzero.loc[:, "climate_model":"year_15T"]
data_sel = data.filter(
    [
        "ms",
        "ensemble_member",
        "average_cdr_between_net0_and_15",
        "max_deployed_CDR",
        "SRM_at_first_crossing",
        "peak_warming",
        "2100_warming",
        "cumulative_emms_net0_to_15",
        "cumulative_pos_emms_at_net0",
        "net_CDR_at_15_temp",
        "TCRE_down",
        "t_above_15",
        "ratios",
        "net-zero_year",
        "TCRE_ratio_up",
        "cumulative_emms_peak_warming_to_15",
    ],
    axis=1,
).reset_index(drop=True)

<IPython.core.display.Javascript object>

Save

In [54]:
data_sel.to_csv(OUTPUT_FILE)

<IPython.core.display.Javascript object>

# Median stats
use dataset where non-net-zero datasets haven't been exlcuded (-> data, not data_T)

In [20]:
median_stats = (
    data.set_index(["ms", "ensemble_member"])[
        [
            "average_cdr_between_net0_and_15",
            "max_deployed_CDR",
            "SRM_at_first_crossing",
            "peak_warming",
            "2100_warming",
            "cumulative_emms_net0_to_15",
            "cumulative_pos_emms_at_net0",
            "net_CDR_at_15_temp",
            "TCRE_down",
        ]
    ]
    .groupby("ms")
    .median()
)

<IPython.core.display.Javascript object>

In [21]:
median_stats.to_csv(OUTPUT_FILE_MEDIAN)

<IPython.core.display.Javascript object>